# 대화형 챗봇이란?

인공지능을 생각할 때 인간의 언어를 이해하고 서로 소통할 수 있는 기계를 자연스럽게 떠올리곤 한다. 그리고 우리 주변에서 볼 수 있는 챗봇은 모두 대화형이 아니다. [챗봇의 5가지 유형](https://tonyaround.com/%ec%b1%97%eb%b4%87-%ea%b8%b0%ed%9a%8d-%eb%8b%a8%ea%b3%84-%ec%b1%97%eb%b4%87%ec%9d%98-5%ea%b0%80%ec%a7%80-%eb%8c%80%ed%91%9c-%ec%9c%a0%ed%98%95-%ec%a2%85%eb%a5%98/)을 보면 대화형, 트리형, 추천형, 시나리오형, 이들을 결합한 결합형 챗봇이 있다. 대화형 챗봇이 아니라면 한계는 명확하게 있다. 진정한 팻봇이면 사용자가 어떤 말을 하더라도 알아듣고 적절하게 대응할 수 있는 자유도가 있어야 하기 때문이다. 그래서 딥러닝을 통한 자연어처리 기술이 가능성을 보여주자, 사람들은 대화형 챗봇이 가져올 변화에 기대감을 가지기 시작했다.   

__챗봇과 딥러닝__   
하지만 우리 주변에 있는 챗봇은 스마트하지 않다. 왜일까? [챗봇 역사의 모든 것](https://blog.performars.com/ko/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5-%EC%B1%97%EB%B4%87chatbot-%EC%B1%97%EB%B4%87-%EC%97%AD%EC%82%AC%EC%9D%98-%EB%AA%A8%EB%93%A0-%EA%B2%83)을 확인해보면 챗봇의 발전 흐름을 볼 수 있다.   
초창기 챗봇에 대한 기대, 한계점을 느낀 후 최근 ALBERT, BERT, ULMFiT, Transformer-XL 등 tansformer 모델을 활용해 pretrain을 적용한 NLP 모델들이 각광을 받고 있다.   

트랜스포머 모델을 기반으로 인코더-디코더 구조를 바탕으로 챗봇을 제작해보자. 물론 좋은 성능을 내기 위해 수많은 코퍼스로 pretrained model을 활용하는게 필요하지만 오늘은 모델의 기본 구조를 알아보는데 초점을 맞추고 알아보자.   

```
$ mkdir -p ~/aiffel/songys_chatbot
```

# 12-2. 트랜스포머와 인코더 디코더

![](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_UcFQAjh.max-800x600.png)

번역기는 인코더와 디코더 두 가지 아키텍처로 구성되어있다. 인코더에 입력문장, 디코더에 출력문장. 이를 훈련하는것은 입력, 출력 두 가지 병렬 구조로 구성된 데이터셋을 훈련한다는 의미다. 이러한 구조는 번역기가 아닌 주어진 질문에 답변할 수 있는 챗봇으로 만들 수 있다.

트랜스포머도 입력 문장을 넣으면 출력 문장을 내뱉는 인코더와 디코더 구성을 갖고 있다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_2_EnQyi4S.max-800x600.png)

초록색이 encoder layer, 핑크색이 decoder layer인데 입력 문장은 누적된 인코더의 층을 통해 정보를 뽑아내고 디코더는 누적된 디코더 층을 통해 출력 문장의 단어를 하나씩 만들어 가는 구조를 갖고 있다. 이 과정을 더 확대하면 다음과 같다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_3_ddZedfW.max-800x600.png)

# 12-3. 트랜스포머의 입력 이해하기

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt
print("슝=3")

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_4_fuzN6PD.png)

많은 자연어 처리 모델들은 텍스트 문장을 입력으로 받기 위해 단어를 임베딩 벡터로 변환하는 벡터화 과정을 거친다. 트랜스포머 또한 그 점에서는 다른 모델들과 다르지 않다. 하지만 트랜스포머 모델의 입력 데이터 처리는 RNN 계열의 모델들과 달리 임베딩 벡터에 어떤 값을 더해준 뒤 입력으로 사용한다.위 그림의 Positional Encoding에 해당하는 부분이다.   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_5_kH52kQN.png)

이렇게 하는 이유는 입력 받을 때 문장에 있는 단어들을 1개씩 순차적으로 받는게 아니라 모든 단어를 한꺼번에 입력으로 받기 때문이다. 트랜스포머와 RNN의 결정적인 차이점이 이 부분이다. RNN은 문장을 구성하는 단어들이 어순대로 모델에 입력되어 어순 정보를 알려줄 필요가 없었는데, 같은 단어라도 어순에 따라 의도하는 정보가 달라질 수 있음을 유의하여 위치를 알려주는 벡터값을 더해 모델의 입력으로 삼는다.   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_6_DyxB6Ax.png)   
$d_{model}$은 임베딩 벡터의 차원을 의미하고, $pos$는 입력 문장에서 입베딩 벡터의 위치를 나타내며, $i$는 임베딩 벡터 내의 차원의 인덱스를 의미한다. 임베딩 행렬과 포지셔널 행렬이라는 두 행렬을 더함으로써 각 단어 벡터에 위치 정보를 더해주게 된다.   
포지셔널 행렬을 확인해보자.

In [ ]:
# 포지셔널 인코딩 레이어
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    # 각도 배열 생성
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스에는 sin 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])
    # 배열의 홀수 인덱스에는 cosine 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    # sin과 cosine이 교차되도록 재배열
    pos_encoding = tf.stack([sines, cosines], axis=0)
    pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
    pos_encoding = tf.reshape(pos_encoding, [position, d_model])

    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

print("슝=3")

In [ ]:
sample_pos_encoding = PositionalEncoding(50, 512)

plt.pcolormesh(sample_pos_encoding.pos_encoding.numpy()[0], cmap='RdBu')
plt.xlabel('Depth')
plt.xlim((0, 512))
plt.ylabel('Position')
plt.colorbar()
plt.show()

행의 크기가 50, 열의 크기가 512인 행렬을 그렸다. 이는 최대 문장의 길이가 50, 임베딩 차원을 512로 하는 모델의 입력 벡터 모양과 같다. 실제 논문에서는 다음과 같이 그림으로 표현하고 있다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_9_l58gVWT.max-800x600.png)

# 12-4. 어텐션

어텐션 메커니즘을 그림으로 표현하면 다음과 같다.   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_10_AaCfqrY.png)   
주어진 query에 대해 모든 key와의 유사도를 각각 구한다. 그리고 구한 유사도를 key와 맵핑되어있는 각각의 값에 반여애준다. 그리고 유사도가 반영된 값을 모두 더해서 뭉쳐주면 어텐션 값이라고 한다.

트랜스포머는 세 가지의 어텐션을 사용한다.   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_11_tFFhFjx.png)   
- 인코더 셀프 어텐션은 인코더에서 인코더의 입력으로 들어간 문장 내 단어들이 서로 유사도를 구한다.
- 디코더 셀프 어텐션은 디코더에서 단어 1개씩 생성하는 디코더가 이미 생성된 앞 단어들과 유사도를 구한다.
- 인코더-디코더 어텐션은 디코더에서 잘 예측하기 위해 인코더에 입력된 단어들과 유사도를 구한다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_12_SIe2V15.png)

트랜스포머 어텐션 함수에 사용되는 query, key, value는 기본적으로 정보를 함축한 단어 벡터이다. 여기서 단어 벡터란 초기 입력으로 사용된 임베딩 벡터가 아닌, 트랜스포머의 여러 연산을 거친 단어 벡터이다. 어텐션중 두개가 셀프 어텐션인데 이의 의미를 알아보자.

셀프 어텐션은 유사도를 구하는 대상이 다른 문장의 단어가 아닌 현재 문장 내의 단어들을 말한다. 위에서 말한 인코더-디코더 어텐션은 서로 다른 단어 목록 사이에서 유사도를 구한다.   
우리는 이것, 저것에 대한 의미를 문장에 있는 단어들로 통해 쉽게 유추한다. 하지만 기계는 그렇지 않다. 그러므로 셀프 어텐션을 통해 유사도를 구하고 이를 통해 무엇과 연관되어 있는지 확률을 구할 수 있다.

# scaled dot product attention

어텐션이 단어들간 유사도를 구하는 메커니즘인데 유사도를 어떻게 구할까?   
$Attention(Q,K,V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$   
$Q,K,V$는 순서대로 query, key, value를 말한다. 어텐션 함수는 query에 대한 모든 key의 유사도를 각각 구한다. 그리고 구한 유사도를 키와 맵핑된 각각의 값에 반영한다. 그리고 유사도가 반영된 값을 모두 더해주면 최종적으로 어텐션 값이 나온다.   
1. Q, K, V는 단어 벡터를 행으로 하는 문장 행렬이다.
2. 백터의 내적(dot product)은 벡터의 유사도를 의미한다.
3. 특정 값을 분모로 사용하는 것은 값의 크기를 조절하는 스케일링을 위함이다.   

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_15_pUfIgKn.png)
문장 행렬 $Q$, 문장 행렬 $K$를 곱하면 초록색 행렬을 얻을 수 있다. 초록생 행렬은 각 단어 벡터의 유사도(내적값)가 모두 기록된 유사도 행렬을 의미한다.   
이 유사도 값을 스케일링 해주기 위해 행렬 전체를 특정 값으로 나누고, 유사도를 0과 1사이의 값으로 normalize하기 위해 softmax 함수를 사용한다. 여기에 문장 행렬 $V$와 곱하면 어텐션 값을 얻게 된다.   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_16_neA52rZ.png)

정리하자면 내적(dot product)를 통해 단어 벡터간 유사도를 구한 후 특정 값을 분모로 나눠주는 방식을 통해 Q, K의 유사도를 구하여 __Scaled Dot Product Attention__ 이라고 한다. 분모에 특정 값을 나눠주지 않았다면 dot product attention이라고 말한다.

In [ ]:
# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
  # 어텐션 가중치는 Q와 K의 닷 프로덕트
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
  if mask is not None:
    logits += (mask * -1e9)

  # softmax적용
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
  output = tf.matmul(attention_weights, value)
  return output

print("슝=3")

# 12-6. 병렬로 어텐션 수행하기

트랜스포머의 `num_heads`라는 하이퍼파라미터 변수는 병렬적으로 몇 개의 어텐션 연산을 수행할지 결정한다.   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_18_nnOTx9p.png)   
트랜스포머의 초기 입력인 문장 행렬의 크기는 문장의 길이를 행으로, `d_model`은 열의 크기로 갖게 된다. 이렇게 입력된 문장 행렬을 `num_heads`의 수만큼 쪼개서 어텐션을 수행하고, 이렇게 얻은 `num_heads`의 개수만큼 어텐션 값 행렬을 하나로 concatenate한다. 위 그림은 `num_heads`가 8개인 경우인데 concatenate 하면서 열의 크기가 `d_model`이 된다.

### 멀티-헤드 어텐션

병렬로 어텐션을 수행하면 어떤 효과를 얻을 수 있을까?   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_19_FwmaA3q.png)

위 그림은 `num_heads` 값이 8일 때, 병렬로 수행되는 어텐션이 서로 다른 셀프 어텐션 결과를 보여준다. 이렇게 어텐션을 병렬로 수행하는 것을 멀티 헤드 어텐션이라고 부른다.

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # Q, K, V에 각각 Dense를 적용
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 병렬 연산을 위해 여러 개 만든다
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 스케일드 닷 프로덕트 어텐션 함수
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 최종 결과에도 Dense를 한 번 더 적용
    outputs = self.dense(concat_attention)

    return outputs
print("슝=3")

# 12-7. 마스킹

특정 값들을 가려 실제 연산에 방해 되지 않도록 하는 것을 마스킹이라고 부른다. 트랜스포머에서는 어텐션을 위해 크게 두 가지 padding masking, look-ahead masking을 사용한다.   

__Padding Masking__   
패딩 토큰을 이용한 마스킹 방법이다. 자연어 처리에서 패딩은 문장의 길이가 서로 다를 때 문장의 길이를 동일하게 해주는 과정에서 짧은 문장에 숫자 0을 채워 길이를 맞춰주는 전처리 방법이다. 주어진 숫자 0은 실제 의미가 있는 단어는 아니여서 실제 어텐션과 같은 연산에서 제외할 필요가 있다. 패딩 마스킹은 숫자 0의 위치를 체크한다.

In [ ]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]
print("슝=3")

In [ ]:
print(create_padding_mask(tf.constant([[1, 2, 0, 3, 0], [0, 0, 0, 4, 5]])))

숫자가 0인 부분을 체크한 벡터를 리턴한다. 결과를 보면 두 정수 시퀀스에 대해 결과가 출력되는데, 숫자 0인 위치에서만 1이 나오고 아닌 위치에서는 0 벡터를 출력한다. 어텐션 연산할 때 패딩 마스킹을 참고하면 숫자 0을 참고하지 않게 할 수 있다.   

__Look-ahead masking__   
순환 신경망, RNN과 트랜스포머는 문장을 입력받을 때 입력받는 방법이 전혀 다르다. RNN은 step의 개념으로 각 step마다 단어 순서대로 입력받는 구조인데 트랜스포머는 문장 행렬을 만들어 한번에 입력받는 구조이다. 그래서 이 특징 때문에 masking이 필요하다.   
자신보다 다음에 나올 단어를 참고하지 않도록 가리는 기법이 look ahead masking 기법이다. Query 단어 뒤에 나오는 Key 단어들에 마스킹 한다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/_.max-800x600.png)

행을 Query 열을 Key로 표현된 행렬이고 빨간색으로 색칠된 부분이 마스킹이다. 실제 어텐션 연산에서 가리는 역할을 하고 어텐션 연산 시에 현재 단어를 기준으로 이전 단어들과 유사도를 구할 수 있다.   

예를들어 Query 단어가 '찾고'이면 여기 있는데 행에 `<s>, <나는>, <행복을>, <찾고>`를 갖고있고 뒤에는 마스킹이 되어있다. 

In [ ]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)
print("슝=3")

In [ ]:
print(create_look_ahead_mask(tf.constant([[1, 2, 3, 4, 5]])))

마스킹과 패딩 마스킹은 별개이므로 숫자 0인 단어가 있으면 패딩을 해야한다. 다음과 같이 적용하자.

In [ ]:
print(create_look_ahead_mask(tf.constant([[0, 5, 1, 5, 5]])))

# 12-8. 인코더

트랜스포머의 인코더 층은 셀프 어텐션과 피드포워드 신경망 총 2개의 sublayer로 구성되어 있다. 셀프 어텐션 레이어는 멀티 헤드 어텐션으로 병렬적으로 이루어져있다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_21_Y7Cy8sm.max-800x600.png)   

두 개의 서브 층을 가지는 하나의 인코더 층을 구현하는 함수를 살펴보자. 함수 내부적으로 첫 번쨰 서브 층과 두 번째 서브 층을 구현하고 있다.

In [ ]:
# 인코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 두 개의 서브 레이어가 존재한다
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })

  # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 두 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
print("슝=3")

![](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_22_teJgoCi.max-800x600.png)

이렇게 구현한 인코더 층을 임베딩 층과 포지서녕 인코딩을 연결하고, 사용자가 원하는 만큼 인코더 층을 쌓으면 트랜스포머의 인코더가 완성된다.   
인코더와 디코더 내부에는 서브층 이후 훈련을 돕는 layer normalization이 사용됐다. 위 그림에서 normalize에 해당하는 부분이다.   
트랜스포머는 num_layers 개수의 인코더 층을 쌓는다. 논문에서는 6개의 인코더 층을 사용했다.

In [ ]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # num_layers만큼 쌓아올린 인코더의 층.
  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
print("슝=3")

# 12-9. 디코더